In [3]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [4]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [5]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://ekitan.com/event/station-2590',
   'title': '東京駅周辺のイベント - 駅探',
   'content': '1. 駅探 2. 東京駅の時刻表・乗り換え 3. # 東京駅周辺のイベント ## 東京駅のイベント一覧 **1〜10件**／80件（新着順） * #### 親子の安心を高める子育て講座～心理学を活かして～ 子どもとのアタッチメントを学ぶ 期間2025年10月20日(月) 会場甲南大学ネットワークキャンパス東京 + 東京都 千代田区 + 東京駅／大手町駅(東京)／三越前駅 1. 無料 2. 屋内 * #### 京都アカデミアフォーラムin丸の内共催「京都新聞講座in東京『京都あれこれ』」 新たな「鬼平犯科帳」ができるまで 期間2025年9月8日(月) 会場京都大学東京オフィス\u3000会議室AB + 東京都 千代田区 + 東京駅／二重橋前駅／大手町駅(東京)駅 1. 無料 2. 屋内 * #### 丸の内ストリートマーケット by Creema 人気のクラフトイベントが6年ぶりに復活！ 期間2025年8月29日(金)～8月30日(土) 会場行幸地下通路 + 東京都 千代田区 + 二重橋前駅／東京駅／大手町駅(東京)駅 期間2025年8月1日(金)～8月24日(日) + 東京駅／二重橋前駅／大手町駅(東京)駅 期間2025年11月16日(日) + 大手町駅(東京)／竹橋駅／東京駅 期間2025年11月1日(土) + 二重橋前駅／東京駅／大手町駅(東京)駅 期間2025年10月4日(土)～10月5日(日) + 二重橋前駅／東京駅／大手町駅(東京)駅 期間2025年8月13日(水) + 東京駅／日本橋駅(東京)／大手町駅(東京)駅 期間2025年8月28日(木) + 東京駅／二重橋前駅／大手町駅(東京)駅 期間2025年8月7日(木) + 東京駅／二重橋前駅／大手町駅(東京)駅 * 京橋(東京) * 大手町(東京) * 日本橋(東京) * 東京メトロ日比谷線 八丁堀駅 徒歩2分 1K / 26.78m2 / 5階 東京メトロ日比谷線 八丁堀駅 徒歩3分 1K / 33.58m2 / 4階 / 築10年 東京メトロ有楽町線 銀座一丁目駅 徒歩2分 ワンルーム / 

In [15]:
# ツール定義
def define_tools():
    #チャットボットは通常これを表示しません
    #print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [7]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [8]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [9]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [10]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。具体的な面積は以下の通りです：

- 東京都の面積：約2,194平方キロメートル
- 沖縄県の面積：約2,271平方キロメートル

したがって、沖縄県は東京都よりも約77平方キロメートル広いです。


In [11]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)


------define_tools(ツール定義)------
最近1ヶ月以内に東京駅で行われるイベントについての情報は以下の通りです。

1. **東京駅でのイベント**  
   - **イベント名**: 東京駅・丸の内のゴールデンウィークイベント  
   - **日付**: 2023年4月29日から5月6日  
   - **詳細**: 東京駅周辺で様々なアクティビティや特設イベントが開催されます。  
   - **URL**: [詳細はこちら](https://www.enjoytokyo.jp/feature/gw/event_area_tokyostation/)

2. **Gransta情報**  
   - **新着情報**: Gransta内のショップでは、特別キャンペーンやイベントが行われています。具体的な内容は公式サイトで確認可能です。  
   - **URL**: [Granstaニュース](https://www.gransta.jp/news/)

3. **八重洲地下街のイベント**  
   - **イベント名**: 60周年祭  
   - **日付**: 6月2日から12月31日まで  
   - **詳細**: 期間中に特別なイベントやキャンペーンが開催され、盛り上がる予定です。  
   - **URL**: [八重洲地下街イベント情報](https://www.yaechika.com/)

これらのイベントは、東京駅及びその周辺で訪れる際の参考になるでしょう。他にも特定の店舗や地区ごとのイベントがあるため、個々の詳細は各ウェブサイトを訪問し確認することをお勧めします。


In [16]:
# チャットボットへの組み込み
tools = define_tools()

teacher_role = "あなたは、質問をする生徒に、彼らのキャリアを導くために親切にアドバイスを与える教師です。"
messages = [{"role": "system", "content": teacher_role},]

while( True ):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")

    # 質問が入力されなければ終了
    if question.strip() == "":
        break

    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問:What are AI events in Tokyo this month?'

Here are some AI-related events happening in and around Tokyo this month:

1. **AIPE 2023**  
   - **Date**: October 20-22, 2023  
   - **Location**: Tokyo, Japan  
   - **Details**: The International Conference on Artificial Intelligence and Power Engineering will be held, focusing on advancements in AI within the power engineering sector.  
   - [More information](https://www.aipe.org/aipe2023.html)

2. **AI EXPO TOKYO [Autumn] 2023**  
   - **Date**: October 25-27, 2023  
   - **Location**: Makuhari Messe, Chiba, Japan  
   - **Details**: This is Japan's largest trade show for AI technology companies, showcasing the latest innovations and technologies in the AI sector.  
   - [More information](https://www.tradeindia.com/tradeshows/112879/ai-expo-tokyo-autumn-2023.html)

3. **AIBC 2023**  
   - **Date**: October 27-29, 2023  
   - **Location**: University of Aizu in Aizu-Wakamatsu, Japan (a bit outside of Tokyo)  
   - **Details**: The AIBC event focuses on blockchain, AI, and relat